In [ ]:
import numpy as np
import librosa
import os
from mutagen.mp3 import MP3
import pickle

In [1]:
def readMfcc(audioPath,size, length):
    featuresArray = []
    for i in range(0, size, length):
        if i + length <= size - 1:
            y, sr = librosa.load(audioPath, offset=i / length, duration=length)  #Instead of"length" used to be 0.1f
            S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
            log_S = librosa.logamplitude(S, ref_power=np.max)
            mfcc = librosa.feature.mfcc(S=log_S, sr=sr, n_mfcc=13)
            featuresArray.append(mfcc)

    return np.reshape(featuresArray,(len(featuresArray),13*5),order='C')

In [1]:
def getLabel(fileName, label_dict):
    genre = label_dict[fileName.split('-')[0]];
    ans = np.zeros(len(label_dict))
    ans[genre] = 1;
    return ans;

In [26]:
def saveDataIn(path, data):
    with open(path, 'wb') as f:
        f.write(pickle.dumps(data))

In [27]:
sample_length = 1000

label_dict ={
    "rock": 0,
    "blues": 1,
    "jazz": 2
}
array = np.array(range(13))

In [ ]:
walk_dir = "/Users/alvarodemarcosales/Desktop/songs"
inputs = []
labels = []
for root, subdirs, files in os.walk(walk_dir):
    for filename in files:
        if filename.endswith("mp3"):
            file_path = os.path.join(root, filename)
            audio = MP3(file_path)
            print("{}:{}".format(int(audio.info.length/60),(int(audio.info.length%60))))
            inputs.append(readMfcc(file_path,int(audio.info.length*1000)), sample_length)
            labels.append(getLabel(filename, label_dict))

In [ ]:
saveDataIn('inputs', data)
saveDataIn('labels', labels)